In [1]:
#Loaded Libraries

library(tidyverse)
library(repr)
library(tidymodels)
library(caret)
install.packages("expss")
library(expss)
library(ggplot2)
library(GGally)
library("gridExtra")


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [2]:
# Data Loaded 

cleveland_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", 
                           col_names = c("age", 
                                         "sex", 
                                         "cp", 
                                         "trestbps", 
                                         "chol", 
                                         "fbs", 
                                         "restercg", 
                                         "thalach", 
                                         "exang", 
                                         "oldpeak", 
                                         "slope", 
                                         "ca", 
                                         "thal", 
                                         "num"))
cleveland_data$num <- as.factor(cleveland_data$num)
head(cleveland_data)

Parsed with column specification:
cols(
  age = col_double(),
  sex = col_double(),
  cp = col_double(),
  trestbps = col_double(),
  chol = col_double(),
  fbs = col_double(),
  restercg = col_double(),
  thalach = col_double(),
  exang = col_double(),
  oldpeak = col_double(),
  slope = col_double(),
  ca = col_character(),
  thal = col_character(),
  num = col_double()
)



age,sex,cp,trestbps,chol,fbs,restercg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0


In [3]:
# see if there is other number rather than 0-4 in the num column
cleveland_data  %>%
    select(num)  %>% 
    unlist() %>% 
    levels()

# transform num from 0-4 to 0 or 1 (1, 2, 3, 4) and store into diagnosis variable
cleveland_data <- cleveland_data %>%
                  mutate(diagnosis = dplyr::recode(num, '0' = 'absent', .default = 'present'))

head(cleveland_data)

# to see how the original data distribute
cleveland_data %>% 
    group_by(diagnosis) %>%
    summarize(n = n())

# We select all the numerical columns and diagnosis column
cleveland_data2 <- cleveland_data%>%
            select(age, trestbps, chol, thalach, oldpeak, diagnosis)
head(cleveland_data2)

heart_tidy <- cleveland_data2

# Above codes are from the proposal, with the tidy dataset cleveland_data2 renamed as heart_tidy

[1] "0" "1" "2" "3" "4"

age,sex,cp,trestbps,chol,fbs,restercg,thalach,exang,oldpeak,slope,ca,thal,num,diagnosis
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>,<fct>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0,absent
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2,present
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1,present
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0,absent
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0,absent
56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0,absent


`summarise()` ungrouping output (override with `.groups` argument)



diagnosis,n
<fct>,<int>
absent,164
present,139


age,trestbps,chol,thalach,oldpeak,diagnosis
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
63,145,233,150,2.3,absent
67,160,286,108,1.5,present
67,120,229,129,2.6,present
37,130,250,187,3.5,absent
41,130,204,172,1.4,absent
56,120,236,178,0.8,absent


In [4]:
set_of_predictors <- c("age","trestbps","chol","thalach","oldpeak")

ps_predictors <- is.element("age",set_of_predictors)

powerset <- function(set) {
    ps <- list()
    set_length <- length(set)
    while (length(ps) < 2 ^ set_length - 1) {
        for(i in 1:set_length) {
            sample_i <- as.list(sample(set,i))
            t <- FALSE
            for(x in ps) {
                if(setequal(sample_i,x)) {
                   t <- TRUE
                }
            }
            if (t != TRUE) {
                ps <- append(ps,list(sample_i))
            }
        }
    }
    return(ps)
}

ps_predictors <- powerset(set_of_predictors)

ps_predictors #All combinations of the predictors




[[1]]
[[1]][[1]]
[1] "oldpeak"


[[2]]
[[2]][[1]]
[1] "trestbps"

[[2]][[2]]
[1] "thalach"


[[3]]
[[3]][[1]]
[1] "trestbps"

[[3]][[2]]
[1] "age"

[[3]][[3]]
[1] "oldpeak"


[[4]]
[[4]][[1]]
[1] "age"

[[4]][[2]]
[1] "chol"

[[4]][[3]]
[1] "trestbps"

[[4]][[4]]
[1] "oldpeak"


[[5]]
[[5]][[1]]
[1] "age"

[[5]][[2]]
[1] "oldpeak"

[[5]][[3]]
[1] "trestbps"

[[5]][[4]]
[1] "thalach"

[[5]][[5]]
[1] "chol"


[[6]]
[[6]][[1]]
[1] "age"

[[6]][[2]]
[1] "chol"


[[7]]
[[7]][[1]]
[1] "oldpeak"

[[7]][[2]]
[1] "chol"

[[7]][[3]]
[1] "trestbps"


[[8]]
[[8]][[1]]
[1] "trestbps"


[[9]]
[[9]][[1]]
[1] "oldpeak"

[[9]][[2]]
[1] "age"


[[10]]
[[10]][[1]]
[1] "thalach"

[[10]][[2]]
[1] "oldpeak"

[[10]][[3]]
[1] "chol"

[[10]][[4]]
[1] "age"


[[11]]
[[11]][[1]]
[1] "oldpeak"

[[11]][[2]]
[1] "chol"


[[12]]
[[12]][[1]]
[1] "thalach"

[[12]][[2]]
[1] "trestbps"

[[12]][[3]]
[1] "age"


[[13]]
[[13]][[1]]
[1] "age"


[[14]]
[[14]][[1]]
[1] "oldpeak"

[[14]][[2]]
[1] "thalach"

[[14]][[3]]
[1] "trestbps"


[[15]]
[[15]][[1]]
[1] "chol"


[[16]]
[[16]][[1]]
[1] "thalach"

[[16]][[2]]
[1] "oldpeak"

[[16]][[3]]
[1] "trestbps"

[[16]][[4]]
[1] "chol"


[[17]]
[[17]][[1]]
[1] "thalach"


[[18]]
[[18]][[1]]
[1] "oldpeak"

[[18]][[2]]
[1] "thalach"


[[19]]
[[19]][[1]]
[1] "trestbps"

[[19]][[2]]
[1] "chol"

[[19]][[3]]
[1] "thalach"


[[20]]
[[20]][[1]]
[1] "age"

[[20]][[2]]
[1] "thalach"

[[20]][[3]]
[1] "trestbps"

[[20]][[4]]
[1] "chol"


[[21]]
[[21]][[1]]
[1] "chol"

[[21]][[2]]
[1] "thalach"


[[22]]
[[22]][[1]]
[1] "oldpeak"

[[22]][[2]]
[1] "thalach"

[[22]][[3]]
[1] "chol"


[[23]]
[[23]][[1]]
[1] "age"

[[23]][[2]]
[1] "thalach"

[[23]][[3]]
[1] "oldpeak"

[[23]][[4]]
[1] "trestbps"


[[24]]
[[24]][[1]]
[1] "trestbps"

[[24]][[2]]
[1] "oldpeak"


[[25]]
[[25]][[1]]
[1] "age"

[[25]][[2]]
[1] "trestbps"

[[25]][[3]]
[1] "chol"


[[26]]
[[26]][[1]]
[1] "chol"

[[26]][[2]]
[1] "trestbps"


[[27]]
[[27]][[1]]
[1] "oldpeak"

[[27]][[2]]
[1] "age"

[[27]][[3]]
[1] "chol"


[[28]]
[[28]][[1]]
[1] "age"

[[28]][[2]]
[1] "trestbps"


[[29]]
[[29]][[1]]
[1] "age"

[[29]][[2]]
[1] "oldpeak"

[[29]][[3]]
[1] "thalach"


[[30]]
[[30]][[1]]
[1] "thalach"

[[30]][[2]]
[1] "age"


[[31]]
[[31]][[1]]
[1] "chol"

[[31]][[2]]
[1] "age"

[[31]][[3]]
[1] "thalach"

In [6]:
# We will use KNN Classification with 10 folds cross-validation for choosing the appropriate k value. The resulting accuracy will be calculated
seed <- 1234
set.seed(seed)
heart_split <- initial_split(heart_tidy, prop = 0.75, strata = diagnosis)
heart_train <- training(heart_split)
heart_test <- testing(heart_split)

# Perform 10 folds Cross-Validation for heart_train dataset to choose appropriate k


heart_10fold <- vfold_cv(heart_train, v=10, strata = diagnosis)


knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbor = tune()) %>%
            set_engine("kknn") %>%
            set_mode("classification")

gridvals <- tibble(neighbors = seq(from = 1, to = 23))

Formula <- c()
K_value <- c()
Accuracy <- c()

for (combination in ps_predictors) {
    y <- "diagnosis ~"
    x <- combination[[1]]
    i <- 1
    while(i < length(combination)) {
        x <- paste(x, combination[[i+1]], sep = " + ") 
        i = i + 1
    }
    heart_formula <- as.formula(paste(y,x))
    
    
    
knn_recipe <- recipe(heart_formula, data = heart_train) %>% # Important !!!!!!!!!!!! Using only 3 predictors (chol,thalach, oldpeak) will gain the max accuracy
            step_center(all_predictors()) %>%
            step_scale(all_predictors())

heart_workflow <- workflow() %>%
                add_model(knn_spec) %>%
                add_recipe(knn_recipe)

heart_result <- heart_workflow %>%
                tune_grid(resamples = heart_10fold, grid = gridvals) %>%
                collect_metrics() 

accuracy <-  heart_result %>%
            filter(.metric == "accuracy") 


# accuracy_versus_k <- ggplot(accuracy, aes(x = neighbors, y = mean))+
#        geom_point() +
#        geom_line() +
#        labs(x = "Neighbors", y = "Accuracy Estimate") +
#        scale_x_continuous(breaks = seq(0, 23, by = 1)) +  # adjusting the x-axis
#        scale_y_continuous(limits = c(0.4, 1.0)) +# adjusting the y-axis 
#        theme(text=element_text(size=20))

# accuracy_versus_k

appropriate_k <- accuracy %>%
                arrange(-mean) %>%
                slice(1) %>%
                select(neighbors) %>%
                pull()

knn_spec_new <- nearest_neighbor(weight_func = "rectangular", neighbor = appropriate_k) %>%
            set_engine("kknn") %>%
            set_mode("classification")

heart_fit <- workflow() %>%
                    add_model(knn_spec_new) %>%
                    add_recipe(knn_recipe) %>%
                    fit(data = heart_train)

heart_predictions <- predict(heart_fit, heart_test) %>%
                    bind_cols(heart_test)

heart_metrics <- heart_predictions %>% 
    metrics(truth = diagnosis, estimate = .pred_class)

heart_conf_mat <- heart_predictions %>%
    conf_mat(truth =diagnosis, estimate = .pred_class)

# print(heart_formula)    
Formula <- append(Formula,heart_formula)
#print(paste("Appropriate k =", appropriate_k))
K_value <- append(K_value, appropriate_k)  
# print(heart_metrics)
accuracy_val <- heart_metrics %>%
            filter(.metric == "accuracy") %>%
            select(.estimate) %>%
            pull()
Accuracy <- append(Accuracy,accuracy_val)
            
# heart_conf_mat
}

In [7]:
Formula <- as.character(Formula)
All_info <- data.frame(Formula, K_value, Accuracy) %>%
            arrange(-Accuracy)

All_info

Formula,K_value,Accuracy
<chr>,<int>,<dbl>
diagnosis ~ oldpeak + thalach + chol,21,0.7333333
diagnosis ~ trestbps + oldpeak,15,0.7333333
diagnosis ~ oldpeak,16,0.7200000
diagnosis ~ thalach + oldpeak + chol + age,23,0.7200000
diagnosis ~ oldpeak + chol,19,0.7200000
diagnosis ~ trestbps + age + oldpeak,17,0.7066667
diagnosis ~ oldpeak + chol + trestbps,23,0.7066667
diagnosis ~ oldpeak + thalach + trestbps,21,0.7066667
diagnosis ~ oldpeak + thalach,23,0.7066667
